In [ ]:
import os
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point
from time import sleep
from tqdm import tqdm
from interaction_process.process_utils.mapinfo import *
from interaction_process.process_utils.datamerge import *
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import gc

### Load some necessary files for data preparation

1. Target vehicle index pickle list: <code>caselist</code> gives all possible case_ids and <code>carid</code> gives all car_ids that have complete 4s seconds. There are 4 possible options: <code>train_index</code>(training), <code>val_index</code>(choose one target vehicle from each case in the validation set), <code>test_index</code>(testing), <code>val_all_index</code>(choose all possible target vehicles from each case in the validation set). 

2. <code>Alane.pkl</code> removes all undrivable lanes from each scenarios

3. <code>laneinformation</code> uses <code>get_all_mapinfo</code> to extract maps/lane information from <code>.osm</code> fiiles. It is a dictionary whose keys are scenario names.

4. <code>vision_</code> defines the minimum vision range for each vehicle. 

In [ ]:
with open('./interaction_data/reference/val_index.pickle', 'rb') as handle:
    s = pickle.load(handle)
caselist, carid = s[0], s[1]

with open('Alane.pkl', 'rb') as f:
    Dnew = pickle.load(f)

laneinformation = get_all_mapinfo('./interaction_data/maps/', segment=5, shred=50)
vision_ = np.ones(17)*12

### Creat the parameter dictionary and define the Dataset object

1. <code>datadir</code> is the position of the corresponding set; <code>max_segment</code> is the maximum number of vectors for each spline, <code>max_distance</code> is the maximum lateral distance.
2. <nana>train=True</code> for both training and validation sets, otherwise <nano>False</nano>.

In [ ]:
paradict = dict(device='cpu', datadir='./interaction_data/val/', mapdir='./interaction_data/maps/', train=False, max_segment=5,
           max_distance=30., vision=vision_, case_list=caselist, car_id=carid, laneinfo=laneinformation, alane=Dnew)
int_dst = InteractionDataset(paradict)

The block below is for generating the training set, to make sure that the memory is enough, we split it into 4 <code>.npz</code> files. 
If the memory of your device is enough, just combine them into one.

In [ ]:
cut = [0,12000, 24000, 36000, 47584]
for c in range(1,5):
    T = []
    M = []
    Y = []
    nb_agents = []
    nb_splines = []
    L = []
    S = []
    A = []
    for i in tqdm(range(cut[c-1], cut[c])):
        for car in carid[i][:8]:
            x1, x2, y, nba, nbs, l, adj, s = int_dst.preprocess(caselist[i], car)
            gc.disable()
            T.append(x1)
            M.append(x2)
            Y.append(y)
            nb_agents.append(nba)
            nb_splines.append(nbs)
            L.append(l)
            S.append(s)
            A.append(adj)
            gc.enable()
    T = np.array(T)
    M = np.array(M)
    Y = np.array(Y).squeeze()
    N_agents = np.array(nb_agents)
    N_splines = np.array(nb_splines)
    L = np.array(L)
    S = np.array(S)
    A = np.array(A)
    np.savez_compressed('./interaction_merge/train'+str(c), trajectory=T, maps=M, lanefeature=L, adjacency=A,
                        nbagents=N_agents, nbsplines= N_splines, intention=Y, lanescore=S)

The two blocks below are for generating the smaller validation set (one target vehicle each case) and the test set.

In [ ]:
T = []
M = []
Y = []
nb_agents = []
nb_splines = []
L = []
S = []
A = []
for i in tqdm(range(len(caselist))):
    x1, x2, y, nba, nbs, l, adj, s = int_dst.preprocess(caselist[i], carid[i])
    gc.disable()
    T.append(x1)
    M.append(x2)
    Y.append(y)
    nb_agents.append(nba)
    nb_splines.append(nbs)
    L.append(l)
    S.append(s)
    A.append(adj)
    gc.enable()
    
T = np.array(T)
M = np.array(M)
Y = np.array(Y).squeeze()
N_agents = np.array(nb_agents)
N_splines = np.array(nb_splines)
L = np.array(L)
S = np.array(S)
A = np.array(A)
np.savez_compressed('./interaction_merge/val', trajectory=T, maps=M, lanefeature=L, adjacency=A,
                    nbagents=N_agents, nbsplines= N_splines, intention=Y, lanescore=S)

In [ ]:
T = []
M = []
nb_agents = []
nb_splines = []
L = []
A = []
Z = []
R = []
for i in tqdm(range(len(caselist))):
    x1, x2, nba, nbs, l, adj, ori, ang = int_dst.preprocess_test(caselist[i], carid[i], i)
    gc.disable()
    T.append(x1)
    M.append(x2)
    L.append(l)
    Z.append(ori)
    nb_agents.append(nba)
    nb_splines.append(nbs)
    R.append(ang)
    A.append(adj)
    gc.enable()
T = np.array(T)
M = np.array(M)
N_agents = np.array(nb_agents)
N_splines = np.array(nb_splines)
L = np.array(L)
Z = np.array(Z)
R = np.array(R)
A = np.array(A)
np.savez_compressed('./interaction_merge/test', trajectory=T, maps=M, adjacency=A, lanefeature=L,
                    nbagents=N_agents, nbsplines= N_splines, radian=R, origin=Z)

The block below is for generating the larger validation set (containing all possible target vehicles in the validation set).

In [ ]:
# generate all possible cases in the validation set
T = []
M = []
Y = []
nb_agents = []
nb_splines = []
L = []
S = []
A = []
for i in tqdm(range(len(caselist))):
    for car in carid[i]:
        x1, x2, y, nba, nbs, l, adj, s = int_dst.preprocess(caselist[i], car)
        gc.disable()
        T.append(x1)
        M.append(x2)
        Y.append(y)
        nb_agents.append(nba)
        nb_splines.append(nbs)
        L.append(l)
        S.append(s)
        A.append(adj)
        gc.enable()
T = np.array(T)
M = np.array(M)
Y = np.array(Y).squeeze()
N_agents = np.array(nb_agents)
N_splines = np.array(nb_splines)
L = np.array(L)
S = np.array(S)
A = np.array(A)
np.savez_compressed('./interaction_merge/valall', trajectory=T, maps=M, lanefeature=L, adjacency=A,
                    nbagents=N_agents, nbsplines= N_splines, intention=Y, lanescore=S)